In [5]:
import numpy as np
import xarray as xr

from eval_utilities import spatial_temporal_metrics as stm
from eval_utilities import visualization as vis
import matplotlib.pyplot as plt



In [6]:
# load ensembles
v1=xr.open_zarr("/data/ch23/data_ch23/euro_mlp_v1_train_2010_2019_val_2020_2020.zarr")
v2=xr.open_zarr("/data/ch23/data_ch23/euro_mlp_v2_train_2010_2019_val_2020_2020.zarr")
v3=xr.open_zarr("/data/ch23/data_ch23/euro_mlp_v3_train_2010_2019_val_2020_2020.zarr")

In [7]:
# load groundtruth

train_ds = xr.open_zarr("/data/ecland_i6aj_o400_2010_2022_6h_euro.zarr").sel(time=slice("2020", "2022"))  

In [8]:
def crps(y_true, y_pred, sample_weight=None, norm=False):
    """Calculate Continuous Ranked Probability Score -- change for our case
    Data based on size (N, np_full) where N=number of samples (in time, lat, lon) and np_full is height (=40) 
    Args:
     * y_true : np.array (N, np_full) ground truth from AD99 for N samples, for profile of height np_full
     * y_pred : np.array (n_seeds, N, np_full) predicted from n_seeds ensembles, for N samples, for profile of height np_full
    Returns:
        np.array
            CRPS score for each height profile (np_full).
    @https://github.com/lm2612/WaveNet_UQ/
    """
    num_samples = y_pred.shape[0]
    y_pred = np.sort(y_pred, axis=0)
    diff = y_pred[1:] - y_pred[:-1]
    weight = np.arange(1, num_samples) * np.arange(num_samples - 1, 0, -1)
    weight = np.expand_dims(weight, (-2,-1))
    ##y_true = np.expand_dims(y_true, 0)
    absolute_error = np.mean(np.abs(y_pred - np.expand_dims(y_true, 0)), axis=(0)) 
    per_obs_crps = absolute_error - np.sum(diff * weight, axis=0) / num_samples**2
    if norm:
        crps_normalized = np.where(np.abs(y_true)> 1E-14, per_obs_crps/np.abs(y_true), np.nan)
        return np.nanmean(crps_normalized, axis=0)
    return np.average(per_obs_crps, axis=0, weights=sample_weight)

In [9]:
def crpss(y_true, y_pred, y_ref):
    """Calculate Continuous Ranked Probability Skill Score
    Args:
     * y_true : np.array (N, np_full) ground truth
     * y_pred : np.array (n_seeds, N, np_full) predicted from model
     * y_ref : np.array (n_seeds, N, np_full) reference prediction
    """
    crps_model = crps(y_true, y_pred)
    crps_ref = crps(y_true, y_ref)
    return 1 - (crps_model / crps_ref)

### PLot ensemble inference time series of one variable at one grid point

In [ ]:
# save the .zarr for all the variables in ./ch23

### Plot CRPS and CRPSS for different variables but for the entire inference time period

### plot CRPS and CRPSS for one variable but for different period of inference time, as well as the confidence interval of the score

### Uncertainty against Error

#### Characteristics of a good ensemble
Forecasts from a good ensemble should: 

display no mean errors (bias); otherwise the probabilities will be biased as well.
exhibit sharpness (i.e. have relatively small spread where the uncertainty is small).
have the ability to span the full climatological range; otherwise the probabilities will either over- or under-forecast the risks of anomalous or extreme weather events.

https://confluence.ecmwf.int/display/FUG/Section+5+Forecast+Ensemble+%28ENS%29+-+Rationale+and+Construction